# Trace blobs (e.g. nuclei, cells, ...) over the z-axis

Code written and conceptualized largely by Nathan De Fruyt (nathan.defruyt@kuleuven.be, nathan.defruyt@gmail.com) with initial leads by Wim Thiels (wim.thiels@kuleuven.be) in the Beets and Jelier labs. 

The algorithm is simple, but functional for minimal goals:
1. **version 3 edit:** instead of blob recognition, I threshold on the percentile of I values (i.e. also intrinsic background correction)
1. **version 4 edit:** splitting blobs that are larger than normal (95% percentile?)
1. **version 5 edit:** still to figure out splitting, but already 3D image labeling + better thresholding
1. **version 6 edit:** segment anything algorithm instead of skimage



* next, I deviated from Wim's advice and went on to work with the blob's 
    1. **center coordinates
    1. **mean intensity value
    1. **radius** (restricted to 20 pixels, as this appeared to be towards the higher end of nucleus radii -- adapt this!)

To this, the program:
1. determines **common blob labels** based on how near they are (max displacement of center = 10 pixels in x/y direction, max rise of 5 planes)
1. renders one line per blob for the plane with the **highest intensity value**

Each step (1. blob identification, 2. blob labelling, 3. summary) are rendered in separate .csv files. 
Blob identification takes the longest (a few hours for a day of pictures). The subseding steps are fast.

**Parameters can therefore be adapted** in the second and third step without consideration. 

Do think about changing parameters to the first step.

___Questions are welcome, optimization of the algorithm too.___

In [1]:
import sys

min_radius = 2
max_radius = 40
threshold = 95
background_threshold = 20
split_ratio = 2/3

# threshold, background_threshold, min_radius, max_radius, split_ratio = list(map(float, sys.argv[1:]))
print(f'Initiating analysis using arguments: \nminimum radius = {min_radius},\nmaximal radius = {max_radius},\nthreshold = {threshold}th percentile,\nbackground is calculated as mean of all values in the {background_threshold}th percentile\nall objects that have a length/width ratio > {split_ratio} will be split in two')

Initiating analysis using arguments: 
minimum radius = 2,
maximal radius = 40,
threshold = 95th percentile,
background is calculated as mean of all values in the 20th percentile
all objects that have a length/width ratio > 0.6666666666666666 will be split in two


In [2]:
## general math and system modules/functions
from math import sqrt, atan, tan, cos, sin
import numpy as np
import os
import glob
import itertools
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
import shutil as sh

## parallellization!
from multiprocessing import Pool, cpu_count

## data formatting!
import pandas as pd

## image import and processing module functions
import czifile as cfile
from skimage import data, data, measure, exposure
from skimage.measure import label, regionprops_table, regionprops
from skimage.morphology import closing
from skimage.segmentation import clear_border
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray
from skimage.filters import gaussian, laplace, threshold_otsu
import cv2 as cv
import PIL
import tifffile as tf
from scipy.spatial.distance import pdist, squareform

import string

## plotting modules
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.pyplot import Axes

## 1. Find data

First I adapted some existing functions to more easily check and handle data either here in jupyter notebook or to the purpose of an application. 

In [3]:
# some easier to handle functions for reading image data
def readczi(filepath):
    img = cfile.imread(filepath)
    return(img[0, 0, :img.shape[2], :img.shape[3], :img.shape[4], 0])

# def showplane(img, z):
#     plt.imshow(img[z, :img.shape[1], :img.shape[2]])
    
# def getplane(img, z):
#     return(img[z, :img.shape[1], :img.shape[2]])

# def getyplane(img, y):
#     return(img[:img.shape[0], :img.shape[1], y])

## 2. Render ___summary___ (mean, max, ...) intensity

#### notes to self: 
1. parallellize?**
1. threshold on min_radius

In [4]:
# def generate_imip(filepath, min_radius, min_z_radius):
#     ## trace back czi file path
#     im_filepath = filepath[:-len('_max_intensity_sum.csv')] + '.czi'
    
#     ## read in czi file
#     imstack = readczi(im_filepath)
    
#     ## 
#     max_intensity_df = pd.read_csv(filepath).drop('Unnamed: 0', axis = 1)
#     max_intensity_df = max_intensity_df.rename(columns = {'x': 'y', 'y': 'x'})
#     max_intensity_df = max_intensity_df[max_intensity_df['label'].isin(max_intensity_df.label[max_intensity_df['r'] > min_radius])] ## 
#     max_intensity_df = max_intensity_df[max_intensity_df['label'].isin(max_intensity_df.label[max_intensity_df['r_z'] > min_z_radius])] ## 
#     # inverted mean intensity projection - DO NOT USE FOR QUANTIFICATION!
    
#     return([os.path.basename(im_filepath), np.max(imstack) - np.mean(imstack, axis=0), max_intensity_df])


**To find the transformed center coordinates of the split object**:

1. We **split** evenly **along the major axis**
1. The new center coordinates follow:
    * $dx = r / sqrt(1+s^{2})$
    
    * $dy = r/sqrt(1+1/s^{2})$
    
    
1. The new areas are circles with the minor axis as radius
1. We include only those pixels that were within the original area

In [5]:
# def extract_img_features(img_bg, img_lbl, bg, split_ratio = 2/3):
#     Idf = pd.DataFrame(regionprops_table(label_image=img_lbl, intensity_image=img_bg, properties = ('label', 'intensity_mean', 'area', 'centroid', 'bbox', 'axis_major_length', 'axis_minor_length', 'orientation')))
#     ## relabel columns
#     Idf = Idf.rename(columns = {'label': "ID", 'intensity_mean': "I", 'area': "area", 'centroid-0': "y", 'centroid-1': "x", 'bbox-0': "y_min", 'bbox-1': "x_min", 'bbox-2': "y_max", 'bbox-3': "x_max", 'axis_major_length': "d_max", 'axis_minor_length': "d_min", 'orientation': 'slope'})
#     ## add a column for ...
#         ## ... r(adius)...
#     Idf.insert(5, 'r', (Idf['d_max'] + Idf['d_min'])/4)
#         ## ...b(ack)g(round)...
#     Idf.insert(2, 'bg', bg)
#         ## ...min(imal circle).area...
#     Idf.insert(4, 'min.area', np.pi*(Idf['d_min']/2)**2)
#         ## ...area.ratio (actual area/minimal area)...
#     Idf['area.ratio'] = Idf['min.area']/Idf['area']
#         ## ...r(adius).ratio (minor axis diameter/major axis diamter)...
#     Idf['r.ratio'] = Idf['d_min']/Idf['d_max']

#         ## ... whether to split the object based on r-ratio and/or min area? 
#     Idf['split'] = 0
#     for i in Idf.index:
#         Idf.loc[i, 'split'] = 1 if ((Idf.loc[i, 'r.ratio'] < split_ratio) | (Idf.loc[i, 'area.ratio'] < split_ratio)) else 0
    
#     ## return dataframe
#     return(Idf)

# def split_objects(ID, Idf, img_lbl):
#     ## shorten name
#     sub = Idf[Idf['ID'] == ID]

#     ## and extract some parameters to make code more readable
#     x = sub.x
#     y = sub.y

#     r = float(sub.d_max)/4
#     s = sub.slope

#     ## calculate deviation of new coordinates from original center coordinates (see cell above for explanation)
#     dx = r/sqrt(1+s**2)
#     dy = r/sqrt(1+ 1/(s**2))

#     ## set new coordinates (subtract and add coordinates to origin coordinates)
#     x1 = float(x - np.sign(s)*dx)
#     x2 = float(x + np.sign(s)*dx)
#     y1 = float(y + np.sign(s)*dy)
#     y2 = float(y - np.sign(s)*dy)

#     origins = [(x1, y1), (x2, y2)]

#     # ## mainly, compose output dataframe:
#     Idf_newlines = Idf.loc[Idf['ID'] == ID, :]

#     ## relabel in image
#     alphabet = list(string.ascii_lowercase) ## to add an a/b to the original label
#     for i, origin in enumerate(origins):
#         ### get origin coordinates
#         x0, y0 = origin

#         ### a function to define whether a coordinate is within the circle (euclidean distance < radius)
#         def in_circle(coord, origin, radius):
#             v = abs(pdist([coord, origin],  metric = 'euclidean')[0])
#             return(v < radius)

#         ### make all possible coordinates within a square around the blob center
#         pxls = list(itertools.product(list(range(int(sub.y_min), int(sub.y_max))), list(range(int(sub.x_min), int(sub.x_max)))))
#         ###  select on those pixels that are within radius from the origin
#             #### all pixels within the bounding box of the object
#         pxls = np.array(pxls)[list(map(lambda c: in_circle(c, (y0, x0), float(sub.d_min)/2), pxls))]
#             #### coordinates of labels for ID 
#         lbl_coords = np.transpose(np.where(img_lbl == ID))
#             #### coordinates of labels to relabel as subsplit object label
#         rlbl_coords = pxls[list(map(lambda pxl: pxl in lbl_coords, pxls))]
#         if len(rlbl_coords) > 0:
#             rows, cols = zip(*rlbl_coords)
#             ### set new label
#             img_lbl[rows, cols]= ID*10 + i + 1
        
#     img_lbl[img_lbl == ID] = 0

#     return(img_lbl)

# def extract_and_split_features_from_img(img, threshold, background_threshold, split_ratio = 2/3, min_radius = 10, max_radius = 30):
    
#     ## I. Threshold image
    
#     ## threshold image on percentile
#     thresh = np.percentile(img, threshold)
#     img_thresh = img*(img > thresh)

#     ## calculate background signal as signal below background_threshold percentile
#     bg = np.mean(img[(img < np.percentile(img, background_threshold))])

#     ## subtract background signal for intenstiy calculations
#     img_bg = img_thresh - bg
#     img_bg[img_bg < 0] = 0
#     img_bw = img > thresh

#     ## smoothen surface
#     img_cls = gaussian(closing(img_bw))

#     # label adhering surfaces as blobs
#     img_lbl = label(img_cls)

#     ## II. extract features ####
    
#     ### 1) extract features from initial labels
# #     print(f'>>> extracting object features')
#     Idf = extract_img_features(img_bg, img_lbl, bg = bg, split_ratio = split_ratio)
#     Idf = Idf[Idf['r'] > min_radius]
#     Idf = Idf[Idf['r'] < max_radius]
# #     print(f">>> found {len(Idf[Idf['split']])} objects that we will split")

#     ### 2) plit objects that are at least 1.5 times longer than they are wide (ratio minor axis/major axis < 2/3 = area using min_radius < 2/3 of actual area)
#     # only if there are objects to split:
#     if len(Idf[Idf['split'] == 1]) > 0:
#         ## iterate over objects to split
#         for ID in Idf['ID'][Idf['split'] == 1]:
#             ## subset data on object ID and relabel img_lbl
#             img_lbl = split_objects(ID, Idf, img_lbl)
# #             print(f">>> split all ({len(Idf[Idf['split']])}) objects, re-calculating features")

#             ## recalcualte features
# #             print(f">>> split object {ID}, re-calculating features")
#             Idf = extract_img_features(img_bg, img_lbl, bg = bg, split_ratio = split_ratio)
#             ## drop old label row
# #             print(f">>> dropping row {Idf.index[Idf['ID'] == ID]}")
#             Idf = Idf.drop(Idf.index[Idf['ID'] == ID])

#     ## III. Return output data
#     Idf.index = range(0, len(Idf.index))
#     Idf = Idf[(Idf['r'] > min_radius)]
#     Idf = Idf[(Idf['r'] < max_radius)]
#     return(Idf, img_bg)

In [6]:
def get_img_and_Idf_from_imstack(imstack, sigma, threshold, split_ratio = 2/3):
    
    ## take laplacian of the gaussian of the image - make sigma wide enough for good smoothing
    img_lp = laplace(gaussian(imstack, sigma = sigma))

    ## threshold on percentile - very strict, yet slightly permissive threshold of 99.8
    thresh = np.percentile(img_lp, threshold)
    # otsu = threshold_otsu(img)
    img_bw = img_lp > thresh

    ## close the edges and label adhering regions
    img_cls = closing(img_bw)
    img_lbl = label(img_cls)

    ## subtract the background from the image (everything that's below the threshold is considered background)
    bg = np.mean(img_lp < thresh)
    img_bg = imstack - bg
    img_bg[img_bg < 0] = 0

    ## now plot
#     img_tp = img_lbl
#     img_tp = np.mean(img_tp, axis = 0) - np.max(img_tp, axis = 0)
#     plt.imshow(img_tp, cmap = 'gray')
    
    ## initiate an output dataframe
#     Idf = pd.DataFrame([], columns = ['ID', 'I', 'bg', 'area', 'x', 'y', 'r', 'd_max', 'd_min', 'z', 'r_z', 'split'])

    ## iterate over all z-planes
#     for z in tqdm(range(0, imstack.shape[0])):
#         ## get intensity data and the background-subtracted, thresholded image
#         sub_Idf, plane = extract_and_split_features_from_img(imstack[z, :, :], threshold = threshold, background_threshold = background_threshold, split_ratio = split_ratio, min_radius = min_radius, max_radius = max_radius)

#         ## add z
#         sub_Idf['z'] = z

#         ## save image and data
#         Idf = pd.concat([Idf, sub_Idf], axis = 0)
#         seq[z, :, :] = plane

    ## measure features
    Idf = pd.DataFrame(regionprops_table(label_image=img_lbl, intensity_image=img_bg, properties = ('label', 'intensity_mean', 'centroid', 'area')))
    
    ## relabel columns
    Idf = Idf.rename(columns = {'label': "ID", 'intensity_mean': "I", 'area': "area", 'centroid-0': "z", 'centroid-1': "y", 'centroid-2': "x"})
    
    ## calculate radius
    Idf['r'] = list(map(lambda x: np.cbrt(3*x/(4*np.pi)), Idf['area']))
#     Idf['ID'] = range(0, len(Idf.index))
#     Idf.index = Idf['ID']
#     Idf['r'] = (Idf['r_x'] + Idf['r_y'])/2
    
    return(Idf, img_bg)
    

In [7]:
# ## to see the blobs
# # def plot_blobs_on_inv_img(Idf, img):
# #     ## inverted LUT image:
# #     img_inv = np.max(img) - img

# #     ## initiate subplots
# #     fig, axes = plt.subplots(1, 1)
    
# #     ## iterate over blobs
# #     for i in range(0, len(Idf)):
# #         ## extract variables
# #         ID, mean, sd, bg, r_x, x, r_y, y, size = Idf.iloc[i]
# #         ## calculate radius
# #         mean_r = np.mean([r_x, r_y])
# #         ## plot the blob + a margin
# #         c = plt.Circle((x, y), mean_r + .5*mean_r, color= 'black', linewidth=.3, linestyle = '--', fill=False)
# #         axes.add_patch(c)
# #         ## show the center of the blob
# #         plt.text(x, y, '+', fontsize = 'xx-small', horizontalalignment='center', verticalalignment='center')
# #         axes.imshow(img_inv, cmap = 'gray')
    
# #     ## show them on the figure
# #     plt.tight_layout()
# #     plt.show()
    
# # ## iterate over all planes

# def merge_overlapping_blobs(df, max_disp = 20, min_radius = 5):
#     # initialise an ID and label column:
#     df['blob_ID'] = df.index

#     # keep track of the z-plane
#     zo = min(df.z.unique())
#     max_rise = max_disp

#     # generate a distance matrix
#     dist = squareform(pdist(np.array(df.loc[:, np.array(['x', 'y', 'z'])], dtype = 'uint16'), 'euclidean'))

#     # and a label dictionary
#     relabel_dict = {}

#     # for each z-plane in the data:
#     # 1. determine the rise since the last z-plane
#     # 2. find blobs that are at a dist < max_disp
#     # 3. put the key-value (ID-label) pair in a dictionary

#     # It is KEY to go progressively through the z-plane (not mapping over all at once).

#     for i, z in enumerate(sorted(df.z.unique())[1:]):
#         # match blob indices in the new plane to blob indices in the previous plane
#         blob_match = {}

#         # get the indices of the rows (in the previous z-plane) and columns (in this z-plane)
#         ids_zo = np.array(df.index[df['z'] < z]) # rows in previous z-plane
#         ids_z = np.array(df.index[df['z'] == z]) # columns in this z-plane

#         # define a function to relabel IDs to merge with close and overlapping ID in previous planes
#         def relabel_blob(ind, ids_zo = ids_zo, max_disp = max_disp, z = z):
#             # subset distance matrix - we compare from the previous plane (zo) to this plane (z)
#             sub = dist[ids_zo, ind]
#             label_ind = ids_zo[np.where([overlaps and closest for overlaps, closest in zip((sub <= max_disp), (sub == np.min(sub)))])[0]]

#             newlabel = df.loc[label_ind[0], 'blob_ID'] if (len(label_ind) > 0) else df.blob_ID[df.index == ind]
#             return(str(int(newlabel)))

#         for ind in ids_z:
# #             print(f'index {str(ind)} will be labelled {str(int(relabel_blob(ind)))}')
#             df.loc[ind, 'blob_ID'] = relabel_blob(ind)
    
#     return(df)

# # then, summarize on only maximum intensity value
# def render_max_intensities_df(merged_labels_df):
# #     print(f'##**Make sure you applied function __merge_overlapping_blobs__ first!**##')
    
#     ## less typing
#     df = merged_labels_df
    
#     ## initiate array with right size
#     max_Idf = np.zeros(shape = (len(df.blob_ID.unique()),len(df.columns)))

#     ## return line with max I for each blob
#     for i, ID in enumerate([x for x in df.blob_ID.unique() if x != "NA"]):
#         sub = df[(df['blob_ID'] == ID)]
#         max_Idf[i, :] = sub[sub['I'] == np.max(sub['I'])]
#         max_Idf[i, np.where(sub.columns == 'r_z')[0][0]] = np.sum(sub['r_z'])/2

#     ## convert array to df and return
#     return(pd.DataFrame(max_Idf, columns = merged_labels_df.columns))

In [8]:
# Idf = df_s
# img = np.mean(imstack, axis = 0)
# ## open labels file
# fig, axes = plt.subplots(1, 1)

# ## normalize intensities
# # rel_Is = map[x/np.max(list(imip[2]['I.mean'])) for x in list(imip[2]['I.mean'])]
# img_inv = np.mean(imstack, axis = 0) - np.max(imstack, axis = 0)
# # plt.imshow(img_inv, cmap = 'gray')

# for i, cell in enumerate(Idf['ID']):
#     x, y, lbl, area, r = Idf.loc[i, np.array(['x', 'y', 'ID', 'area', 'r'])]
# #     color = 'red' if area > np.pi*r_min**2 else 'black'

#         ## if you like to plot, use below code on an image
#     plt.text(y = y, x = x, s = '+', color = 'black', fontsize = 'small', horizontalalignment='center',
#      verticalalignment='center')
# #     ## plot the blob + a margin
#     c = plt.Circle((x, y),  r*1.6, color= 'black', linewidth=.3, linestyle = '--', fill=False)
#     axes.add_patch(c)
    
# axes.imshow(img_inv, cmap = 'gray')
# # axes.set_axis_off()

# plt.tight_layout()
# plt.show()

In [9]:
# folder = filedialog.askdirectory()
# files = glob.glob(folder + '/*.czi')


In [10]:
# file = files[0]

In [11]:
# ## print so we know that reading the file worked
# # print(f'# Processing file {i+1}/{len(files)} ({os.path.basename(file)})')

# ## read file
# imstack = readczi(file)
# print(f' >>> read stack with {imstack.shape[0]} planes.')

# ## fetch blobs from image
# df, seq = get_img_and_Idf_from_imstack(imstack, sigma = sigma, threshold = threshold)
# print(f' >>> found {df.shape[0]} nuclei (?) in image stack')

# ## save as .csv file
# # out_filename = folder + '/' + os.path.basename(file)[:-4] + '_objects.csv'
# # df.to_csv(out_filename)
# print(f' >>> written blobs to disk')


In [12]:
# df

In [13]:
## don't touch this cell!!
sigma = 7
threshold = 99.8

folder = filedialog.askdirectory()
files = glob.glob(folder + '/*.czi')
    
for i, file in enumerate(files):
    ## print so we know that reading the file worked
    print(f'# Processing file {i+1}/{len(files)} ({os.path.basename(file)})')
    
    ## read file
    imstack = readczi(file)
    print(f' >>> read stack with {imstack.shape[0]} planes.')
    
    ## fetch blobs from image
    df, seq = get_img_and_Idf_from_imstack(imstack, sigma = sigma, threshold = threshold)
    print(f' >>> found {df.shape[0]} nuclei (?) in image stack')
    
    ## save as .csv file
    out_filename = folder + '/' + os.path.basename(file)[:-4] + '_objects.csv'
    df.to_csv(out_filename)
    print(f' >>> written blobs to disk')

#     ## merge the blobs to a single intensity value (blobs are merged on how near they are)
#     ## for so long that the dataframe gets compacter (shorter)
#     c0 = len(df.index) ## initiate initial 'compactness'
#     compacter = True ## set value to check whether it got better or not
#     Idf = df
#     while compacter == True:
#         df_m = merge_overlapping_blobs(Idf, max_disp = 30)
#         df_s = render_max_intensities_df(df_m)
#         c = len(df_s.index)

#         if c == c0:
#             print(f'--- no more overlapping blobs')
#             df_s = render_max_intensities_df(df_s)
#             compacter = False
#         else:
#             print(f'--- discovered {c0 - c} more overlapping blobs')
#             c0 = c
#             Idf = df_s

#     print(f' >>> rendered intensity data')

#     ## save as .csv file
#     df_filename = folder + '/' + os.path.basename(file)[:-4] + '_labeled_blobstack_v4.csv'
#     df_s.to_csv(df_filename)
#     print(f' >>> written intensity data to disk\n\n')

    ## save tif of background subtracted frames
#     seq_filename = folder + '/' + os.path.basename(file)[:-4] + '_seq_v3.tif'
#     tf.imsave(seq_filename, seq)
#     print(' >>> wrote background subtracted image sequence to disk')

# Processing file 1/16 (20240820-PXXX_vX-IBE954_21_1.czi)
 >>> read stack with 176 planes.


KeyboardInterrupt: 